In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold, StratifiedKFold, train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score, confusion_matrix, roc_curve, precision_recall_curve, auc, f1_score, cohen_kappa_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.feature_selection import RFE, SelectFromModel, RFECV
from xgboost import XGBClassifier, plot_importance
from IPython import display
from datetime import datetime

import matplotlib.pyplot as plt
from matplotlib import cm

from pylab import rcParams

import os
import re
import time
import warnings
warnings.filterwarnings('ignore')

seed = 9001
np.random.seed(seed)

### Read in processed extracted 6 hour window data training and test set

In [ ]:
# training set
X_all = pd.read_csv('6hour_Xtrain.csv')
y_all = pd.read_csv('6hour_ytrain.csv')

# test set
X_test = pd.read_csv('6hour_Xtest.csv')
y_test = pd.read_csv('6hour_ytest.csv')

### Define threshold that gives best F1-score

In [ ]:
def performance2(y, y_pred, print_ = 1, *args):   
    """ Calculate performance measures for a given ground truth classification y and predicted 
    probabilities y_pred. If *args is provided a pre-defined threshold is used to calculate the performance.
    If not, the threshold giving the best mean sensitivity and specificity is selected. The AUC is calculated
    for a range of thresholds using the metrics package from sklearn. """

    # xx and yy values for ROC curve
    fpr, tpr, thresholds = roc_curve(y, y_pred, pos_label=1)
    # area under the ROC curve
    AUC = auc(fpr, tpr)

    # xx and yy values for AUPR
    precision, recall, thresholds = precision_recall_curve(y, y_pred, pos_label=1)
    # convert to f1-score
    fscore = (2*precision*recall) / (precision+recall)
    fscore[np.isnan(fscore)] = 0

    # if a threshold is specified, use it
    if args:
        threshold = args[0]
    else:
    # if a threshold is not specified, we will choose the threshold that gives the best f1-score
        ix = np.argmax(fscore)
        threshold = thresholds[ix]        
        
    # transform the predicted probability into a binary classification
    y_pred[y_pred >= threshold] = 1
    y_pred[y_pred < threshold] = 0
    
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
        
    # basic accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    # for auc
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    # for f1-score
    precision1 = tp/(tp+fp)
    recall1 = sensitivity # detection rate
    
    # for false alarm rate
    fpr2 = fp/(fp+tn) # distinguish with false positive rate from calculating AUC
    fnr = fn/(fn+tp)
    
    # f1-score
    f1 = max(fscore)

    # kappa score
    kappa = cohen_kappa_score(y, y_pred)

    
    # print the performance and plot the ROC curve    
    if print_ == 1:
        print('Threshold: ' + str(round(threshold, 2)))
        print('TP: ' + str(tp))
        print('TN: ' + str(tn))
        print('FP: ' + str(fp))
        print('FN: ' + str(fn))
        
        print("Accuracy: " + str(round(accuracy, 2)))
        print('Sensitivity: ' + str(round(sensitivity, 2)))
        print('Specificity: ' + str(round(specificity, 2)))
        print('Precision: ' + str(round(precision1, 2)))
        print('Recall (Detection rate): ' + str(round(recall1, 2)))
        print('F1-score: ' + str(round(f1, 2)))
        print('AUC: ' + str(round(AUC, 2)))
        print('FPR (False alarm rate): ' + str(round(fpr2, 2)))
        print('FNR: ' + str(round(fnr, 2)))
        print('Kappa: ' + str(round(kappa, 2)))
    
        plt.figure(figsize = (4,3))
        plt.scatter(x = fpr, y = tpr, label = None)
        plt.plot(fpr, tpr, label = 'Classifier', zorder = 1)
        plt.plot([0, 1], [0, 1], 'k--', label = 'Random classifier')
        plt.scatter(x = 1 - specificity, y = sensitivity, c = 'black', label = 'Operating point', zorder = 2)
        plt.legend()
        plt.xlabel('1 - specificity')
        plt.ylabel('sensitivity')
        plt.show()
        

    return threshold, accuracy, sensitivity, specificity, AUC, precision1, recall1, f1, fpr2, fnr, kappa

### Create function to evaluate model performance on training and test set maximizing F1-score

In [ ]:
def model_evaluation2(model, X_train, y_train, X_test, y_test, print_):
    
    # tune - parameter estimation 
    print('TRAINING SET')
    y_pred_prob_train = model.predict_proba(X_train)
    threshold, accuracy_tr, sensitivity_tr, specificity_tr, AUC_tr, precision_tr, \
    recall_tr, f1_tr, fpr_tr, fnr_tr, kappa_tr = performance2(y_train, np.delete(y_pred_prob_train, 0, 1), print_) # retain the probabilities of positive class only

    # test
    print('TEST SET')
    y_pred_prob_test = model.predict_proba(X_test)
    _, accuracy_test, sensitivity_test, specificity_test, AUC_test, precision_test, \
     recall_test, f1_test, fpr_test, fnr_test, kappa_test = performance2(y_test, np.delete(y_pred_prob_test, 0, 1), print_, threshold)
    
    # save the results
    results_train = pd.DataFrame(data = [[threshold, accuracy_tr, sensitivity_tr, specificity_tr, AUC_tr,
                                          precision_tr, recall_tr, f1_tr, fpr_tr, fnr_tr, kappa_tr, X_train.shape[1]]],
                                 columns = ['Threshold','Accuracy', 'Sensitivity', 'Specificity', 'AUROC',
                                            'Precision', 'Recall', 'F1', 'FPR (False Alarm)', 'FNR', 'Kappa', '# features'])

    results_test = pd.DataFrame(data = [[threshold, accuracy_test, sensitivity_test, specificity_test, AUC_test,
                                          precision_test, recall_test, f1_test, fpr_test, fnr_test, kappa_test, X_train.shape[1]]],
                                columns = ['Threshold','Accuracy', 'Sensitivity', 'Specificity', 'AUROC',
                                           'Precision', 'Recall', 'F1', 'FPR (False Alarm)', 'FNR', 'Kappa', '# features'])
        
    return results_train, results_test, y_pred_prob_train, y_pred_prob_test

### Create a stratified cv (to preserve class distribution during Cross-validation)

In [ ]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# 50/50 Ratio

### Create balanced data using Random Undersampling

In [ ]:
rus = RandomUnderSampler(sampling_strategy='majority', random_state=seed)

In [ ]:
X_uds, y_uds = rus.fit_resample(X_all, y_all)

In [ ]:
print('Sepsis patterns after Random Under-sampling:', np.count_nonzero(y_uds == 1))
print('Non-sepsis patterns after Random Under-sampling:', np.count_nonzero(y_uds == 0))

# 80/20 Ratio

In [ ]:
ratio8020 = RandomUnderSampler(sampling_strategy=1/4, random_state=seed)

In [ ]:
X_8020, y_8020 = ratio8020.fit_resample(X_all, y_all)

In [ ]:
print('Sepsis patterns for 80/20 ratio:', np.count_nonzero(y_8020 == 1))
print('Non-sepsis patterns for 80/20 ratio:', np.count_nonzero(y_8020 == 0))

# 90/10 Ratio

In [ ]:
ratio9010 = RandomUnderSampler(sampling_strategy=1/9, random_state=seed)

In [ ]:
X_9010, y_9010 = ratio9010.fit_resample(X_all, y_all)

In [ ]:
print('Sepsis patterns for 90/10 ratio:', np.count_nonzero(y_9010 == 1))
print('Non-sepsis patterns for 90/10 ratio:', np.count_nonzero(y_9010 == 0))

# 95/5 Ratio

In [ ]:
ratio9505 = RandomUnderSampler(sampling_strategy=5/95, random_state=seed)

In [ ]:
X_9505, y_9505 = ratio9505.fit_resample(X_all, y_all)

In [ ]:
print('Sepsis patterns for 95/05 ratio:', np.count_nonzero(y_9505 == 1))
print('Non-sepsis patterns for 95/05 ratio:', np.count_nonzero(y_9505 == 0))

# 98/02: original dataset

In [ ]:
print('Sepsis patterns for original ratio:', np.count_nonzero(y_all == 1))
print('Non-sepsis patterns for original ratio:', np.count_nonzero(y_all == 0))

# 2. Random Forest

### Random Undersampling with All variables

In [ ]:
# create the model/estimator
rf_uds = RandomForestClassifier(random_state=seed,
                                oob_score=True,
                                # class_weight='balanced',
                                n_jobs=-1)

In [ ]:
# create a parameters grid to do GridSearch
param_rf = {'n_estimators': [100, 500, 1000], # number of trees in the forest
            'max_depth' : [10, 15, 20],
            'min_samples_leaf': [1, 3, 5]}

In [ ]:
grid_rf_uds = GridSearchCV(estimator=rf_uds,
                           param_grid=param_rf,
                           cv=kfold,
                           scoring='f1',
                           n_jobs=-1) # use all processors

In [ ]:
start = datetime.now()

grid_rf_uds.fit(X_uds, y_uds)

print('Training took: ', datetime.now()-start)

In [ ]:
print('Best: %f using %s' % (grid_rf_uds.best_score_, grid_rf_uds.best_params_))

In [ ]:
mean_f1 = grid_rf_uds.cv_results_['mean_test_score']
std_f1 = grid_rf_uds.cv_results_['std_test_score']
params = grid_rf_uds.cv_results_['params']

In [ ]:
for mean, stdev, param in zip(mean_f1, std_f1, params): # zip: create tuples of value pairs
    print('%f (+-%f) with: %r' % (mean, stdev, param))

In [ ]:
best_rf_uds = grid_rf_uds.best_estimator_

### Performance on training and test sets

In [ ]:
results_train_rf_uds_all, results_test_rf_uds_all, \
y_pred_prob_train_rf_uds, y_pred_prob_test_rf_uds = model_evaluation2(best_rf_uds, 
                                                                      X_uds, y_uds,
                                                                      X_test, y_test,
                                                                      print_ = 1)

### Save results to dataframe

In [ ]:
rf_results_train = pd.DataFrame()
rf_results_test = pd.DataFrame()

In [ ]:
rf_results_train = rf_results_train.append(results_train_rf_uds_all.rename(index={results_train_rf_uds_all.index[-1]: 'RF 50/50 Ratio All features'}))
rf_results_test = rf_results_test.append(results_test_rf_uds_all.rename(index={results_test_rf_uds_all.index[-1]: 'RF 50/50 Ratio All features'}))

### Random Under-sampling with Feature selection by Recursive Feature Elimination (RFE)

### Take a look at the feature importance by Random Forest

In [ ]:
grid_rf_uds.best_params_

In [ ]:
rf_uds_fs = RFECV(estimator=best_rf_uds, step=1, cv=kfold, scoring='f1', n_jobs=-1)

In [ ]:
start = datetime.now()

rf_uds_fs.fit(X_uds, y_uds)

print('Training took: ', datetime.now()-start)

### Optimal Number of features

In [ ]:
print('Optimal number of features: {}'.format(rf_uds_fs.n_features_))

### Plot the optimal number of features against F1-score

In [ ]:
plt.figure(figsize=(16, 9))
plt.title('Recursive Feature Elimination with Cross-Validation', fontsize=18, fontweight='bold', pad=20)
plt.xlabel('Number of features selected', fontsize=14, labelpad=20)
plt.ylabel('F1-Score', fontsize=14, labelpad=20)
plt.plot(range(1, len(rf_uds_fs.grid_scores_) + 1), rf_uds_fs.grid_scores_, color='#303F9F', linewidth=3)

### Now only select the important features according to the model

In [ ]:
rfe_feat = rf_uds_fs.support_

### Use these selected features for Random Undersample dataset

In [ ]:
X_uds_rfe, y_uds_rfe = rus.fit_resample(X_all.loc[:, rfe_feat], y_all)

In [ ]:
X_all.loc[:, rfe_feat]

In [ ]:
rfe_features = X_all.loc[:, rfe_feat].columns

In [ ]:
rfe_features

### Model

In [ ]:
start = datetime.now()

best_rf_uds.fit(X_uds_rfe, y_uds_rfe)

print('Training took: ', datetime.now()-start)

### Feature importance determined by the model

In [ ]:
rcParams['figure.figsize'] = 10, 10
features = rfe_features
importances = best_rf_uds.feature_importances_
indices = np.argsort(importances)

plt.title('Feature importance of each variable in Random Forest model')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')

### Model Performance

In [ ]:
results_train_rf_uds_rfe, results_test_rf_uds_rfe, \
y_pred_prob_train_rf_rfe, y_pred_prob_test_rf_rfe = model_evaluation2(best_rf_uds, 
                                                                      X_uds_rfe, y_uds_rfe,
                                                                      X_test.loc[:, rfe_feat], y_test,
                                                                      print_ = 1)

### Save results to dataframe

In [ ]:
rf_results_train = rf_results_train.append(results_train_rf_uds_rfe.rename(index={results_train_rf_uds_rfe.index[-1]: 'RF 50/50 Ratio with Feature selection'}))
rf_results_test = rf_results_test.append(results_test_rf_uds_rfe.rename(index={results_test_rf_uds_rfe.index[-1]: 'RF 50/50 Ratio with Feature selection'}))

# 80/20 Ratio: All features

In [ ]:
grid_rf_uds.best_params_

In [ ]:
# create the model/estimator
rf_8020 = RandomForestClassifier(random_state=seed,
                                 oob_score=True,
                                #  class_weight='balanced',
                                 n_jobs=-1,
                                 max_depth=20,
                                 min_samples_leaf=1,
                                 n_estimators=1000)

In [ ]:
start = datetime.now()

rf_8020.fit(X_8020, y_8020)

print('Training took: ', datetime.now()-start)

### Performance on training and test set

In [ ]:
results_train_rf_8020, results_test_rf_8020, \
y_pred_prob_train_rf_8020, y_pred_prob_test_rf_8020 = model_evaluation2(rf_8020, 
                                                                        X_8020, y_8020,
                                                                        X_test, y_test,
                                                                        print_ = 1)

### Save results to dataframe

In [ ]:
rf_results_train = rf_results_train.append(results_train_rf_8020.rename(index={results_train_rf_8020.index[-1]: 'RF 80/20 Ratio All features'}))
rf_results_test = rf_results_test.append(results_test_rf_8020.rename(index={results_test_rf_8020.index[-1]: 'RF 80/20 Ratio All features'}))

# 80/20 Ratio: Feature selection

In [ ]:
# create the model/estimator
rf_8020_rfe = RandomForestClassifier(random_state=seed,
                                     oob_score=True,
                                     # class_weight='balanced',
                                     n_jobs=-1,
                                     max_depth=20,
                                     min_samples_leaf=1,
                                     n_estimators=1000)

### Use the selected features for 80/20 dataset

In [ ]:
X_8020_rfe, y_8020_rfe = ratio8020.fit_resample(X_all.loc[:, rfe_feat], y_all)

### Model

In [ ]:
start = datetime.now()

rf_8020_rfe.fit(X_8020_rfe, y_8020_rfe)

print('Training took: ', datetime.now()-start)

### Model Performance

In [ ]:
results_train_rf_8020_rfe, results_test_rf_8020_rfe, \
y_pred_prob_train_rf_8020_rfe, y_pred_prob_test_rf_8020_rfe = model_evaluation2(rf_8020_rfe, 
                                                                                X_8020_rfe, y_8020_rfe,
                                                                                X_test.loc[:, rfe_feat], y_test,
                                                                                print_ = 1)

### Save results to dataframe

In [ ]:
rf_results_train = rf_results_train.append(results_train_rf_8020_rfe.rename(index={results_train_rf_8020_rfe.index[-1]: 'RF 80/20 Ratio with Feature Selection'}))
rf_results_test = rf_results_test.append(results_test_rf_8020_rfe.rename(index={results_test_rf_8020_rfe.index[-1]: 'RF 80/20 Ratio with Feature Selection'}))

# 90/10 Ratio: All features

In [ ]:
grid_rf_uds.best_params_

In [ ]:
# create the model/estimator
rf_9010 = RandomForestClassifier(random_state=seed,
                                 oob_score=True,
                                #  class_weight='balanced',
                                 n_jobs=-1,
                                 max_depth=20,
                                 min_samples_leaf=1,
                                 n_estimators=1000)

In [ ]:
start = datetime.now()

rf_9010.fit(X_9010, y_9010)

print('Training took: ', datetime.now()-start)

### Performance on training and test set

In [ ]:
results_train_rf_9010, results_test_rf_9010, \
y_pred_prob_train_rf_9010, y_pred_prob_test_rf_9010 = model_evaluation2(rf_9010, 
                                                                        X_9010, y_9010,
                                                                        X_test, y_test,
                                                                        print_ = 1)

### Save results to dataframe

In [ ]:
rf_results_train = rf_results_train.append(results_train_rf_9010.rename(index={results_train_rf_9010.index[-1]: 'RF 90/10 Ratio All features'}))
rf_results_test = rf_results_test.append(results_test_rf_9010.rename(index={results_test_rf_9010.index[-1]: 'RF 90/10 Ratio All features'}))

# 90/10 Ratio: Feature selection

In [ ]:
# create the model/estimator
rf_9010_rfe = RandomForestClassifier(random_state=seed,
                                     oob_score=True,
                                     # class_weight='balanced',
                                     n_jobs=-1,
                                     max_depth=20,
                                     min_samples_leaf=1,
                                     n_estimators=1000)

### Use the selected features for 90/10 dataset

In [ ]:
X_9010_rfe, y_9010_rfe = ratio9010.fit_resample(X_all.loc[:, rfe_feat], y_all)

### Model

In [ ]:
start = datetime.now()

rf_9010_rfe.fit(X_9010_rfe, y_9010_rfe)

print('Training took: ', datetime.now()-start)

### Model Performance

In [ ]:
results_train_rf_9010_rfe, results_test_rf_9010_rfe, \
y_pred_prob_train_rf_9010_rfe, y_pred_prob_test_rf_9010_rfe = model_evaluation2(rf_9010_rfe, 
                                                                                X_9010_rfe, y_9010_rfe,
                                                                                X_test.loc[:, rfe_feat], y_test,
                                                                                print_ = 1)

### Save results to dataframe

In [ ]:
rf_results_train = rf_results_train.append(results_train_rf_9010_rfe.rename(index={results_train_rf_9010_rfe.index[-1]: 'RF 90/10 Ratio with Feature Selection'}))
rf_results_test = rf_results_test.append(results_test_rf_9010_rfe.rename(index={results_test_rf_9010_rfe.index[-1]: 'RF 90/10 Ratio with Feature Selection'}))

# 95/05 Ratio: All features

In [ ]:
grid_rf_uds.best_params_

In [ ]:
# create the model/estimator
rf_9505 = RandomForestClassifier(random_state=seed,
                                 oob_score=True,
                                #  class_weight='balanced',
                                 n_jobs=-1,
                                 max_depth=20,
                                 min_samples_leaf=1,
                                 n_estimators=1000)

In [ ]:
start = datetime.now()

rf_9505.fit(X_9505, y_9505)

print('Training took: ', datetime.now()-start)

### Performance on training and test set

In [ ]:
results_train_rf_9505, results_test_rf_9505, \
y_pred_prob_train_rf_9505, y_pred_prob_test_rf_9505 = model_evaluation2(rf_9505, 
                                                                        X_9505, y_9505,
                                                                        X_test, y_test,
                                                                        print_ = 1)

### Save results to dataframe

In [ ]:
rf_results_train = rf_results_train.append(results_train_rf_9505.rename(index={results_train_rf_9505.index[-1]: 'RF 95/05 Ratio All features'}))
rf_results_test = rf_results_test.append(results_test_rf_9505.rename(index={results_test_rf_9505.index[-1]: 'RF 95/05 Ratio All features'}))

# 95/05 Ratio: Feature selection

In [ ]:
# create the model/estimator
rf_9505_rfe = RandomForestClassifier(random_state=seed,
                                     oob_score=True,
                                     # class_weight='balanced',
                                     n_jobs=-1,
                                     max_depth=20,
                                     min_samples_leaf=1,
                                     n_estimators=1000)

### Use the selected features for 95/05 dataset

In [ ]:
X_9505_rfe, y_9505_rfe = ratio9505.fit_resample(X_all.loc[:, rfe_feat], y_all)

### Model

In [ ]:
start = datetime.now()

rf_9505_rfe.fit(X_9505_rfe, y_9505_rfe)

print('Training took: ', datetime.now()-start)

### Model Performance

In [ ]:
results_train_rf_9505_rfe, results_test_rf_9505_rfe, \
y_pred_prob_train_rf_9505_rfe, y_pred_prob_test_rf_9505_rfe = model_evaluation2(rf_9505_rfe, 
                                                                                X_9505_rfe, y_9505_rfe,
                                                                                X_test.loc[:, rfe_feat], y_test,
                                                                                print_ = 1)

### Save results to dataframe

In [ ]:
rf_results_train = rf_results_train.append(results_train_rf_9505_rfe.rename(index={results_train_rf_9505_rfe.index[-1]: 'RF 95/05 Ratio with Feature Selection'}))
rf_results_test = rf_results_test.append(results_test_rf_9505_rfe.rename(index={results_test_rf_9505_rfe.index[-1]: 'RF 95/05 Ratio with Feature Selection'}))

# 98/02 Ratio: All features

In [ ]:
grid_rf_uds.best_params_

In [ ]:
# create the model/estimator
rf_9802 = RandomForestClassifier(random_state=seed,
                                 oob_score=True,
                                #  class_weight='balanced',
                                 n_jobs=-1,
                                 max_depth=20,
                                 min_samples_leaf=1,
                                 n_estimators=1000)

In [ ]:
start = datetime.now()

rf_9802.fit(X_all, y_all)

print('Training took: ', datetime.now()-start)

### Performance on training and test set

In [ ]:
results_train_rf_9802, results_test_rf_9802, \
y_pred_prob_train_rf_9802, y_pred_prob_test_rf_9802 = model_evaluation2(rf_9802, 
                                                                        X_all, y_all,
                                                                        X_test, y_test,
                                                                        print_ = 1)

### Save results to dataframe

In [ ]:
rf_results_train = rf_results_train.append(results_train_rf_9802.rename(index={results_train_rf_9802.index[-1]: 'RF 98/02 Ratio All features'}))
rf_results_test = rf_results_test.append(results_test_rf_9802.rename(index={results_test_rf_9802.index[-1]: 'RF 98/02 Ratio All features'}))

# 98/02 Ratio: Feature selection

In [ ]:
# create the model/estimator
rf_9802_rfe = RandomForestClassifier(random_state=seed,
                                     oob_score=True,
                                     # class_weight='balanced',
                                     n_jobs=-1,
                                     max_depth=20,
                                     min_samples_leaf=1,
                                     n_estimators=1000)

### Use the selected features for 98/02 dataset

In [ ]:
X_9802_rfe, y_9802_rfe = X_all.loc[:, rfe_feat], y_all

### Model

In [ ]:
start = datetime.now()

rf_9802_rfe.fit(X_9802_rfe, y_9802_rfe)

print('Training took: ', datetime.now()-start)

### Model Performance

In [ ]:
results_train_rf_9802_rfe, results_test_rf_9802_rfe, \
y_pred_prob_train_rf_9802_rfe, y_pred_prob_test_rf_9802_rfe = model_evaluation2(rf_9802_rfe, 
                                                                                X_9802_rfe, y_9802_rfe,
                                                                                X_test.loc[:, rfe_feat], y_test,
                                                                                print_ = 1)

### Save results to dataframe

In [ ]:
rf_results_train = rf_results_train.append(results_train_rf_9802_rfe.rename(index={results_train_rf_9802_rfe.index[-1]: 'RF 98/02 Ratio with Feature Selection'}))
rf_results_test = rf_results_test.append(results_test_rf_9802_rfe.rename(index={results_test_rf_9802_rfe.index[-1]: 'RF 98/02 Ratio with Feature Selection'}))

### Save the training and test dataframes to csv files

In [ ]:
rf_results_train.to_csv('RF_Train_Results.csv', index=True)
rf_results_test.to_csv('RF_Test_Results.csv', index=True)